# Gallery Example: M[2]/M[2]/1 Multi-Class Queue

This example demonstrates a multi-class M/M/1 queue:
- **Arrivals**: Two independent Poisson processes
- **Service**: Exponential service times (same server, different rates)
- **Classes**: 2 job classes with different characteristics
- **Servers**: 1 server serving both classes
- **Scheduling**: FCFS (First-Come-First-Served)

This model is useful for analyzing systems with heterogeneous workloads.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mm1_multiclass(): Network {    """Create M[2]/M[2]/1 multi-class queueing model"""    model = Network("M[2]/M[2]/1")        # Block 1: nodes    source = Source(model, "mySource")    queue = Queue(model, "myQueue", SchedStrategy.FCFS)    sink = Sink(model, "mySink")        # Block 2: classes    oclass1 = OpenClass(model, "myClass1")    source.setArrival(oclass1, Exp(1))      # λ₁ = 1    queue.setService(oclass1, Exp(4))       # μ₁ = 4, so service time = 0.25        oclass2 = OpenClass(model, "myClass2")    source.setArrival(oclass2, Exp(0.5))    # λ₂ = 0.5    queue.setService(oclass2, Exp(4))       # μ₂ = 4, so service time = 0.25        # Block 3: topology - serial routing for both classes    P = model.initRoutingMatrix()    P.addRoute(oclass1, source, queue, 1.0)    P.addRoute(oclass1, queue, sink, 1.0)    P.addRoute(oclass2, source, queue, 1.0)    P.addRoute(oclass2, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_mm1_multiclass()

## Theoretical Analysis for Multi-Class M/M/1

For M[2]/M[2]/1 with:
- **Class 1**: λ₁ = 1, μ₁ = 4, ρ₁ = 0.25
- **Class 2**: λ₂ = 0.5, μ₂ = 4, ρ₂ = 0.125
- **Total**: λ = 1.5, ρ_total = 0.375

Under FCFS, both classes experience the same mean response time:
- **Mean Response Time**: W = 1/(μ_eff - λ_total) where μ_eff depends on class mix

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)

In [ ]:
// Analyze class-specific performance metrics
println("\n=== Class-Specific Analysis ===")
// Get detailed metrics for each class
val solver = MVA(model)
val avg_table = solver.avgTable
// Extract performance metrics by class
println("Performance by class:")
for class_idx in range(model.get_number_of_classes()):
    val class_name = f"Class {class_idx + 1}"
    val queue_row = 1  # Queue is at index 1 (after source)
// Note: The avg_table structure may vary - adjust indices as needed
    println(f"\n{class_name}:")
    println(f"  Queue metrics: {avg_table.iloc[queue_row]}")

In [ ]:
// Compare with single-class systems
println("\n=== Comparison with Single-Class Systems ===")
// Create equivalent single-class systems
fun create_single_class_mm1(arrival_rate, service_rate, class_name): Network {
    val model_single = Network(f"M/M/1-{class_name}")
    val source = Source(model_single, "Source")
    val queue = Queue(model_single, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_single, "Sink")
    
    val oclass = OpenClass(model_single, class_name)
    source.setArrival(oclass, Exp(arrival_rate))
    queue.setService(oclass, Exp(service_rate))
    
    val P = model_single.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_single.link(P)
    
    return model_single
// Analyze individual class performance in isolation
val model1_single = create_single_class_mm1(1.0, 4, "Class1-Isolated")
val model2_single = create_single_class_mm1(0.5, 4, "Class2-Isolated")

val solver1 = MVA(model1_single)
val solver2 = MVA(model2_single)

val avg_table1 = solver1.avgTable
val avg_table2 = solver2.avgTable

println("Class 1 in isolation:")
println(avg_table1)

println("\nClass 2 in isolation:")
println(avg_table2)

println("\nNote: In multi-class FCFS, classes influence each other"s performance")
println("compared to their isolated single-class equivalents.")